# SEL635/SEL5724 - Visão Computacional Aplicada a Robótica



**Aluno:** Preencher com seu nome

**NUSP:** Preencher com seu número USP

Versão do documento: 1.1

# Lista 3 - Calibração de Câmera

O propósito deste exercício é implementar um dos métodos de calibração do livro Trucco e Verri (1998).

## Preparação do ambiente e definição de funções de exercícios anteriores

In [ ]:
# Importar as bibliotecas que serão utilizadas
import numpy as np
import matplotlib.pyplot as plt
import math

Defina a seguir as funções implementadas em exercícios anteriores que serão utilizadas nessa Tarefa:

In [ ]:
def proj_perspectiva(Pc,f):
  (M,N) = np.shape(Pc) # recupera o tamanho da matriz (N linhas, M colunas)
  if(M!=3):
    print('Aviso: a matriz Pc usada como entrada não possui três linhas!')

  pc = np.zeros((3,N))  # Inicializa o vetor pc de dimensão 3xN com zeros

  # Encontra a projeção perspectiva dos pontos no plano da imagem em [mm]:
  ######
  # Coloque aqui o seu código para calcular a matriz pc



  ######

  return pc

In [ ]:
def camera_to_image(pc,sx,sy,ox,oy):
  (M,N) = np.shape(pc) # recupera o tamanho da matriz (N linhas, M colunas)
  if(M!=3):
    print('Aviso: a matriz pc usada como entrada não possui três linhas!')

  p_im = np.zeros((2,N))  # Inicializa o vetor p de dimensão 2xN com zeros

  # Converte as coordenadas dos pontos pc do sistema da câmera em [mm] para o sistema de coordenadas da imagem em [px]:
  ######
  # Coloque aqui o seu código para calcular a matriz p_im #####



  ######

  return p_im

In [ ]:
def calibration_pattern(n,d):
    Pw = np.zeros((3,3*n*n+7))
    Pw[:,0] = [0, 0, 0]
    Pw[:,1] = [n*d, 0, 0]
    Pw[:,2] = [0, n*d, 0]
    Pw[:,3]=[0, 0, n*d];
    Pw[:,4]=[n*d, n*d, 0];
    Pw[:,5]=[n*d, 0, n*d];
    Pw[:,6]=[0, n*d, n*d];

    for i in range(n):
        for j in range(n):
            Pw[:,n*i+j+7]= [d*j, d*i, 0]
            Pw[:,n*n + n*i+j+7]= [0, d*j, d*i];
            Pw[:,2*n*n + n*i+j+7]= [d*i, 0, d*j];

    return Pw

In [ ]:
def Rx(theta):
    theta = np.radians(theta)
    R = np.array([[1, 0, 0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])
    return R
def Ry(theta):
    theta = np.radians(theta)
    R = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    return R
def Rz(theta):
    theta = np.radians(theta)
    R = np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])
    return R
def Homogenea(R,t):
    t = t.reshape(-1, 1) #vetor de translação (formato de vetor coluna)
    H = np.concatenate((R,t),axis=1) #concatena na horizontal a matriz de rotação e vetor de translação
    temp_v = np.array([0,0,0,1]).reshape(1, -1) #ultima linha da matriz homogênia (formato de vetor linha)
    H = np.concatenate((H,temp_v),axis=0) #concatena na vertical a ultima linha da matriz de homografia
    return H

In [ ]:
def world_to_camera(Pw,Rwc,Twc):
    (M,N) = np.shape(Pw) # recupera o tamanho da matriz (M linhas, N colunas)

    # Coloque seu codigo aqui




    return Pc

In [ ]:
def world_to_image(Pw,Rwc,Twc,f,sx,sy,ox,oy):

    # Coloque seu codigo aqui




    return p_im

In [ ]:
def plot_image(p_im,size_x,size_y):
    plt.figure()
    plt.plot(p_im[0,4:],p_im[1,4:],'bo',mfc='none')
    plt.plot(p_im[0,0],p_im[1,0],'ko',mfc='none')
    plt.plot(p_im[0,1],p_im[1,1],'co',mfc='none')
    plt.plot(p_im[0,2],p_im[1,2],'ro',mfc='none')
    plt.plot(p_im[0,3],p_im[1,3],'go',mfc='none')
    plt.axis([0, size_x, size_y, 0])

## Parte A: Imagens do Padrão de Calibração

Considere as duas câmeras e o padrão de calibração definidos e posicinados de acordo com as instruções da Tarefa 2.

Coloque a seguir a sequência de código utilizada ao final da Tarefa 2 para obter a imagem da câmera 1 e da câmera 2 do padrão de calibração.

In [ ]:
## Coloque seu código aqui para fazer cada um dos passos a seguir

# Definir o padrão de calibração:


# Definir a câmera 1:


# TODO: Faça um print dos parâmetros intrínsecos e extrínsecos da camera 1


# Obter a imagem do padrão de calibração a partir da câmera 1:


# Definir a câmera 2:


# TODO: Faça um print dos parâmetros intrínsecos e extrínsecos da camera 2


# Obter a imagem do padrão de calibração a partir da câmera 2:



## Parte B: Calibração de Câmeras

1. Implemente a função `Rwc,Twc,fx,fy,ox,oy = calibrate_camera(Pw,p_im)` utilizando um dos métodos de calibração de câmera apresentado em aula. A entrada da função deve ser uma matriz $P_w$ de dimensão $3\times N$ que são as coordenadas dos pontos do padrão de calibração descritas no sistema de coordenadas do mundo, $\{W\}$, e uma matriz $p_{im}$ de dimensão $2\times N$ que são as coordenadas da imagem em pixels correspondentes a cada um dos pontos do padrão de calibração. A função deve retornar os seguintes parâmetros na ordem: ${^wR_c}$ a orientação da câmera em relação ao sistema de coordenadas do mundo; $^wT_c$ a posição do centro óptico da câmera em relação ao sistema de coordenadas do mundo em [m]; $f_x$ e $f_y$ a distância focal em $[mm/pixel]$; $o_x$ e $o_y$ a posição na imagem da projeção do centro óptico dada em $[pixel]$.

In [ ]:
def calibrate_camera(Pw,p_im):



  return Rwc,Twc,fx,fy,ox,oy

2. Faça um script para testar a calibração para a câmera 1.

3. Faça um script para comparar os parâmetros obtidos através do método de calibração implementado com os parâmetros conhecidos das câmeras. Ou seja, apresente o erro entre os parâmetros `Rwc,Twc,fx,fy,ox,oy` conhecidos e os estimados para a câmera 1

3. Faça um script para testar a calibração para a câmera 2.

4. Faça um script para comparar os parâmetros obtidos através do método de calibração implementado com os parâmetros conhecidos das câmeras. Ou seja, apresente o erro entre os parâmetros `Rwc,Twc,fx,fy,ox,oy` conhecidos e os estimados para a câmera 2.

### Imagens esperadas do Padrão de Calibração

A Figura 1 e 2 mostram as imagens do padrão de calibração esperadas para a câmera 1 e 2 respectivamente.


<div>
<img src='https://drive.google.com/uc?id=15N_OAMHxABu1OYHaJ8puM-NosodEKoKw' width="500"/>
</div>
Figura 1 - Imagem do padrão de calibração obtida pela câmera 1


<div>
<img src='https://drive.google.com/uc?id=1N_bHV5sYUH2W3VXgDLSSOs0ILKOqcqz2' width="500"/>
</div>
Figura 2 - Imagem do padrão de calibração obtida pela câmera 2
